# Deliverable 3. Create a Travel Itinerary Map.
----
1. Import the vacation CSV file. 
2. From the map above with pick 4 cities that are in close proximity (on the same continent) that the customer would travel to and create a directions layer map.
3. For the "travel_mode" use either "DRIVING", "BICYCLING", or "WALKING".
4. Take a screenshot of the route and save it. 
5. Then create a marker layer for the four cities. Each city will should have a pop-up marker that contains
    - The hotel name
    - The city
    - The country
    - The current weather description and the maximum temperature
---

## Make sure the initial dependencies and the Google API key are imported.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config import g_key

In [2]:
# Configure gmaps
gmaps.configure(api_key=g_key)

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [3]:
# Read the WeatherPy_vacation.csv into a DataFrame
input_file = os.path.join("..", "Vacation_Search", "WeatherPy_vacation.csv")
vacation_df = pd.read_csv(input_file)

# Display sample data
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,3,Sibolga,ID,74.12,heavy intensity rain,1.7427,98.7792,Hotel Syariah CN Darussalam
1,5,Jonesboro,US,74.89,clear sky,35.8423,-90.7043,Hilton Garden Inn Jonesboro
2,6,Castro,BR,71.64,broken clouds,-24.7911,-50.0119,CHACARA BAILLY
3,10,Kapaa,US,79.12,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
4,12,Avarua,CK,77.05,broken clouds,-21.2078,-159.7750,Paradise Inn


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [ ]:
# In case of an empty DataFrame, load the sample data provided
# clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
# clean_hotel_df.head(10)

## In this step, you will set-up the pop-up markers. Review the code to create a marker layer map of the vacation search results. This code is the same as in Deliverable 2.

In [4]:
# Review the formatting template
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [5]:
# Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

# Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

## From the vacation search map, *choose four cities* that a customer might want to visit. They should be close together and in the same country.

In [6]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# The starting and ending city should be the same city.
# Cities: Pierre, Saint Anthony, Hamilton, Jonesboro, Kirkwood
vacation_start = vacation_df.loc[vacation_df["City"] == "Pierre", :]
vacation_end = vacation_df.loc[vacation_df["City"] == "Saint Anthony", :]
vacation_stop1 = vacation_df.loc[vacation_df["City"] == "Hamilton", :]
vacation_stop2 = vacation_df.loc[vacation_df["City"] == "Jonesboro", :]
vacation_stop3 = vacation_df.loc[vacation_df["City"] == "Kirkwood", :]

## Use the `to_numpy()` function and list indexing to write code to retrieve the latitude-longitude pairs as tuples from each city DataFrame.

In [7]:
# Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start.loc[:, ["Lat", "Lng"]].to_numpy()
end = vacation_end.loc[:, ["Lat", "Lng"]].to_numpy()
stop1 = vacation_stop1.loc[:, ["Lat", "Lng"]].to_numpy()
stop2 = vacation_stop2.loc[:, ["Lat", "Lng"]].to_numpy()
stop3 = vacation_stop3.loc[:, ["Lat", "Lng"]].to_numpy()

start = start[0]
end = end[0]
stop1 = stop1[0]
stop2 = stop2[0]
stop3 = stop3[0]

## Use the [gmaps documentation](https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html#directions-layer) to create a directions layer map using the variables from the previous step. Where the starting and ending city are the same city, the `waypoints` are the three other cities, and the `travel_mode` is either `"DRIVING"`, `"BICYCLING"`, or `"WALKING"`.

In [9]:
# Define a new figure object
fig = gmaps.figure()

# Create a direction layer map using the start and end latitude-longitude pairs, and stop1, stop2, and stop3 as the waypoints.
# The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
vacation_itinerary =  gmaps.directions_layer(start, end, waypoints=[stop1, stop2, stop3], travel_mode="DRIVING")

# Add the layer to the map
fig.add_layer(vacation_itinerary)

# Display the map
fig

Figure(layout=FigureLayout(height='420px'))

## Use the provided `concat()` function code snippet to combine the four separate city DataFrames into one DataFrame.

In [10]:
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat(
    [
        vacation_start,
        vacation_stop1,
        vacation_stop2,
        vacation_stop3
    ],
    ignore_index = True
)

# Display sample data
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,440,Pierre,US,74.05,overcast clouds,44.3683,-100.3510,Ramkota Hotel
1,571,Hamilton,US,72.19,clear sky,39.1834,-84.5333,Six Acres Bed & Breakfast
2,5,Jonesboro,US,74.89,clear sky,35.8423,-90.7043,Hilton Garden Inn Jonesboro
3,302,Kirkwood,US,74.53,few clouds,38.5834,-90.4068,"Holiday Inn St Louis SW - Route 66, an IHG Hotel"


## Refactor the code from Step 6 to create a new marker layer map of the cities on the travel route.

In [12]:
# Review the formatting template
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [13]:
# Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Define a new figure object
fig = gmaps.figure()

# Add the layer to the map
fig.add_layer(marker_layer)

# Display the map
fig

Figure(layout=FigureLayout(height='420px'))